# TEST NETWORK

## Imports 

In [85]:
import math
import numpy as np
from random import randint

## Classes

In [86]:
class Matrix:
    def __init__(self, rows: int, cols: int, data: list[list[float]] = None):
        self.rows = rows
        self.cols = cols
        self.data = data if data else [[0.0] * cols for _ in range(rows)]

    def shape(self):
        print(f"({self.rows}, {self.cols})")
        
    def show(self):
        [print(row) for row in self.data]

In [87]:
class Layer:
    def __init__(self, previous_size: int, size: int):
        self.nb_neurons = size
        self.W = Matrix(size, previous_size)
        self.B = Matrix(size, 1)
        self.Z = Matrix(size, 1)
        self.A = Matrix(size, 1)

    def show(self):
        print(f"nb_neurons: {self.nb_neurons}")
        print("W:", end='')
        self.W.shape()
        self.W.show()
        print("B:", end='')
        self.B.shape()
        self.B.show()
        print("Z:", end='')
        self.Z.shape()
        self.Z.show()
        print("A:", end='')
        self.A.shape()
        self.A.show()

In [88]:
class Neural_Network:
    def __init__(self, nb_layers: int, layers: list = []):
        self.nb_layers = nb_layers
        self.layers = layers

    def show(self):
        print(f"nb_layers: {self.nb_layers}")
        for layer in self.layers:
            layer.show()

## Functions : Matrix

In [89]:
def addScalar(matrix: Matrix, scalar: float) -> Matrix:
    result = Matrix(matrix.rows, matrix.cols)
    for i in range(matrix.rows):
        for j in range(matrix.cols):
            result.data[i][j] = matrix.data[i][j] + scalar
    return result

In [90]:
def dotScalar(scalar: float, matrix: Matrix) -> Matrix:
    result = Matrix(matrix.rows, matrix.cols)
    for i in range(matrix.rows):
        for j in range(matrix.cols):
            result.data[i][j] = matrix.data[i][j] * scalar
    return result

In [91]:
def subScalar(scalar: float, matrix: Matrix) -> Matrix:
    result = Matrix(matrix.rows, matrix.cols)
    for i in range(matrix.rows):
        for j in range(matrix.cols):
            result.data[i][j] = scalar - matrix.data[i][j]
    return result

In [92]:
def addMatrix(matrix1: Matrix, matrix2: Matrix) -> Matrix:
    result = Matrix(matrix1.rows, matrix1.cols)
    for i in range(matrix1.rows):
        for j in range(matrix1.cols):
            result.data[i][j] = matrix1.data[i][j] + matrix2.data[i][j]
    return result

In [93]:
def subMatrix(matrix1: Matrix, matrix2: Matrix) -> Matrix:
    result = Matrix(matrix1.rows, matrix1.cols)
    for i in range(matrix1.rows):
        for j in range(matrix1.cols):
            result.data[i][j] = matrix1.data[i][j] - matrix2.data[i][j]
    return result

In [94]:
def dotMatrix(matrix1: Matrix, matrix2: Matrix) -> Matrix:
    result = Matrix(matrix1.rows, matrix2.cols)
    for i in range(matrix1.rows):
        for j in range(matrix2.cols):
            for k in range(matrix1.cols):
                result.data[i][j] += matrix1.data[i][k] * matrix2.data[k][j]
    return result

In [95]:
def funcMatrix(matrix: Matrix, func) -> Matrix:
    result = Matrix(matrix.rows, matrix.cols)
    for i in range(matrix.rows):
        for j in range(matrix.cols):
            result.data[i][j] = func(matrix.data[i][j])
    return result

In [96]:
def transposeMatrix(matrix: Matrix) -> Matrix:
    result = Matrix(matrix.cols, matrix.rows)
    for i in range(matrix.rows):
        for j in range(matrix.cols):
            result.data[j][i] = matrix.data[i][j]
    return result

In [97]:
def copyMatrix(matrix: Matrix) -> Matrix:
    result = Matrix(matrix.rows, matrix.cols)
    for i in range(matrix.rows):
        for j in range(matrix.cols):
            result.data[i][j] = matrix.data[i][j]
    return result

In [98]:
def sumMatrix(matrix: Matrix) -> float:
    result = 0
    for i in range(matrix.rows):
        for j in range(matrix.cols):
            result += matrix.data[i][j]
    return result

In [99]:
def sumMatrixAxis(matrix: Matrix, axis: int) -> Matrix:
    result = Matrix(1, matrix.cols) if axis == 0 else Matrix(matrix.rows, 1)
    for i in range(matrix.rows):
        for j in range(matrix.cols):
            if axis == 0:
                result.data[0][j] += matrix.data[i][j]
            else:
                result.data[i][0] += matrix.data[i][j]
    return result

## Functions : Network

In [100]:
def load_nn(path: str) -> Neural_Network:
    f =  open(path, 'r')

    line = f.readline()

    if line.startswith('dataIn'):
        previous_size = int(line.split(' ')[1])
        line = f.readline()

    if line.startswith('Layer'):
        layer = Layer(previous_size, 1)
        network = Neural_Network(int(line.split(' ')[1]) + 1, [layer])

    for _ in range(network.nb_layers - 1):
        line = f.readline()

        size = int(line.split(' ')[1])
        layer = Layer(previous_size, size)

        for index in range(size):
            line = f.readline()
            sep = line.find('|') + 1
            
            layer.B.data[index][0] = float(line[0:sep - 1])
            token = [float(x) for x in line[sep+1:-2].split(', ')]

            for j in range(len(token)):
                layer.W.data[index][j] = token[j]

        previous_size = size
        network.layers.append(layer)

    f.close()
    return network

In [101]:
def save_nn(path: str, network: Neural_Network) -> None:
    f =  open(path, 'w')

    f.write(f"dataIn {network.layers[0].W.cols}\n")
    f.write(f"Layer {network.nb_layers - 1}\n")
    
    for layer in network.layers[1:]:
        f.write(f"Neuron {layer.nb_neurons}\n")

        for index in range(layer.nb_neurons):
            weights = ", ".join(str(weight) for weight in layer.W.data[index])
            write = f"{layer.B.data[index][0]}|[{weights}]\n" 
            f.write(write)

    f.close()

In [102]:
def initiate_nn(network: Neural_Network) -> None:
    for layer in network.layers[1:]:
        for i in range(layer.W.rows):
            for j in range(layer.W.cols):
                layer.W.data[i][j] = randint(0, 100) / 100
            layer.B.data[i][0] = randint(0, 100) / 100

In [103]:
def create_network(nb_layers: int, nb_neurons: list[int], input_size: int) -> Neural_Network:
    layer = Layer(input_size, 1)
    network = Neural_Network(nb_layers + 1, [layer])

    nb_neurons.insert(0, input_size)

    for i in range(nb_layers):
        layer = Layer(nb_neurons[i], nb_neurons[i + 1])
        network.layers.append(layer)

    initiate_nn(network)
    return network

## Function : Activation

In [104]:
def sigmoid(x: float) -> float:
    return 1 / (1 + math.exp(-x))

In [105]:
def sigmoid_derivative(x: float) -> float:
    return sigmoid(x) * (1 - sigmoid(x))

In [106]:
def softmax(matrix: Matrix) -> Matrix:
    result = Matrix(matrix.rows, matrix.cols)
    for i in range(matrix.rows):
        sum = 0.0
        for j in range(matrix.cols):
            sum += math.exp(matrix.data[i][j])
        for j in range(matrix.cols):
            result.data[i][j] = math.exp(matrix.data[i][j]) / sum
    return result

## Exec : Network

In [107]:
def feedforeward(network: Neural_Network, data: Matrix) -> Matrix:
    
    input_layer = network.layers[0]
    input_layer.A = data
    
    X = input_layer.A

    for i in range(1, network.nb_layers):
        layer = network.layers[i]

        layer.W.shape()
        X.shape()

        layer.Z = dotMatrix(layer.W, X)
        layer.A = funcMatrix(layer.Z, sigmoid)
        X = layer.A

    return X

In [108]:
def update(Layer: Layer, dW: Matrix, dB: Matrix, learning_rate: float) -> None:
    Layer.W = subMatrix(Layer.W, dotScalar(learning_rate, dW)) 
    Layer.B = subMatrix(Layer.B, dotScalar(learning_rate, dB))

In [109]:
def back_propagation(network: Neural_Network, data: Matrix, expected: int, learning_rate: float) -> None:
    
    
    m = data.cols
    C = network.nb_layers - 1


    dZ = subScalar(expected, network.layers[C].A)

    for i in range(network.nb_layers - 1, 0, -1):

        dW = dotScalar(1/m, dotMatrix(dZ, transposeMatrix(network.layers[i - 1].A)))
        dB = dotScalar(1/m, sumMatrixAxis(dZ, 1))

        if i > 1:

            dZ = dotMatrix(transposeMatrix(network.layers[i].W), dZ)
            dZ = dotMatrix(dZ, network.layers[i - 1].A)
            sp = dotScalar(-1, network.layers[i - 1].A)
            dZ = dotMatrix(dZ, funcMatrix(sp, sigmoid_derivative))

        update(network.layers[i], dW, dB, learning_rate)


## Main

In [110]:
def deep_neural_network(path: str, data: Matrix, expected: int, iter: int, learning_rate: float) -> None:

    network = create_network(3, [2, 2, 1], 2)

    feedforeward(network, data)

    for _ in range(iter):
        back_propagation(network, data, expected, learning_rate)

In [111]:
def predict(network: Neural_Network, data: Matrix) -> Matrix:

    return feedforeward(network, data)

## Run

In [112]:
iter = 100

X = Matrix(2, 1, [[0], [0]])

network = create_network(3, [2, 2, 1], 2)
predict(network, X).show()

deep_neural_network("", X, 0, 1, 0.01)


(2, 2)
(2, 1)
(2, 2)
(2, 1)
(1, 2)
(2, 1)
[0.6406381312808671]
(2, 2)
(2, 1)
(2, 2)
(2, 1)
(1, 2)
(2, 1)
